# Slice BRATS' data with SimpleITK

## Import libraries

In [1]:
import glob

!pip install tqdm
from tqdm import tqdm 

import numpy as np

!pip install SimpleITK
import SimpleITK as sitk

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached SimpleITK-2.2.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (52.7 MB)


## Load 3D volumes

In [2]:
flairPath="./BRATS/*_flair.nii.gz"
segmentationPath="./BRATS/*_seg.nii.gz"
flair_all=glob.glob(flairPath)
flair_all.sort()

seg_all=glob.glob(segmentationPath)
seg_all.sort()
nb_display=3 # Number of name files to display
print( f" -> First {nb_display} train items ")
[print(flair_all[i], seg_all[i]) for i in range(nb_display)]

print( f" -> Last {nb_display} train items ")
[print(flair_all[i], seg_all[i]) for i in range(len(flair_all)-nb_display,len(flair_all))]

 -> First 3 train items 
./BRATS/BraTS20_Training_001_flair.nii.gz ./BRATS/BraTS20_Training_001_seg.nii.gz
./BRATS/BraTS20_Training_002_flair.nii.gz ./BRATS/BraTS20_Training_002_seg.nii.gz
./BRATS/BraTS20_Training_003_flair.nii.gz ./BRATS/BraTS20_Training_003_seg.nii.gz
 -> Last 3 train items 
./BRATS/BraTS20_Training_367_flair.nii.gz ./BRATS/BraTS20_Training_367_seg.nii.gz
./BRATS/BraTS20_Training_368_flair.nii.gz ./BRATS/BraTS20_Training_368_seg.nii.gz
./BRATS/BraTS20_Training_369_flair.nii.gz ./BRATS/BraTS20_Training_369_seg.nii.gz


[None, None, None]

## Extract the slices

In [3]:
threshold=0.01 #treshold du pourcentage de pixel segmenté minimum dans une image
trainPath="Database/Train/"
testPath="Database/Test/"

for nImage in tqdm(range(len(flair_all))):
    Zs=[]
    stat=sitk.StatisticsImageFilter()
    #Lire l'image
    flair = sitk.ReadImage(flair_all[nImage])
    seg = sitk.ReadImage(seg_all[nImage])
    
    seg =  sitk.Cast(seg,sitk.sitkUInt8)
    size = list(flair.GetSize())
    totalPixel=size[1]*size[0]
    tresholdValue=totalPixel*threshold
    
    
    #Count pixels in 1 image out of 5
    for z in range(0,size[2],5):
         stat.Execute(seg[:,:,z])
         Zs.append([stat.GetSum(),z])
    Zs.sort(reverse=True) 
    size[2]=0 #Because we only want 2D slices from the extractor
    
    #Extract slices with most segmented pixels
    for index in range(min(5,len(Zs))):
        if Zs[index][0]<tresholdValue:
            break
        #Extract slice
        Extractor = sitk.ExtractImageFilter() 
        indexExtrac = [0, 0, Zs[index][1]]
        Extractor.SetSize(size)
        Extractor.SetIndex(indexExtrac)
        flair_slice=Extractor.Execute(flair) 
        
        #Transform slice to save it as PNG
        flair_slice=sitk.RescaleIntensity(flair_slice,0,255)
        flair_slice = sitk.Cast(flair_slice,sitk.sitkUInt8)
        
        #Save slice and divide it between train set and test set
        if nImage+1<=300:
            sitk.WriteImage(flair_slice, trainPath+f"{(nImage+1):03d}"+"_"+str(index+1)+"flair.png")
            sitk.WriteImage(Extractor.Execute(seg), trainPath+f"{(nImage+1):03d}"+"_"+str(index+1)+"seg.png")
        else:
            sitk.WriteImage(flair_slice, testPath+f"{(nImage+1):03d}"+"_"+str(index+1)+"flair.png")
            sitk.WriteImage(Extractor.Execute(seg), testPath+f"{(nImage+1):03d}"+"_"+str(index+1)+"seg.png")

100%|██████████| 369/369 [00:53<00:00,  6.86it/s]
